In [3]:


### Ścieżki wejścia/wyjścia dla modułu "Quality Metrics"

from pathlib import Path

PROJECT_ROOT = Path(".").resolve()

# katalog z obrazami
IMAGE_ROOT = PROJECT_ROOT / "inputs"   # jeżeli masz inną ścieżkę, zostaw swoją
print("IMAGE_ROOT:", IMAGE_ROOT, "| istnieje:", IMAGE_ROOT.exists())

# katalog artefaktów jakości (opcjonalny: miniatury, wykresy)
OUTPUT_DIR = PROJECT_ROOT / "outputs" /"csv"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# wspólny katalog na CSV całego pipeline'u
OUTPUT_CSV_DIR = PROJECT_ROOT / "outputs" / "csv"
OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)

# finalny CSV z metrykami jakości
QUALITY_CSV = OUTPUT_CSV_DIR / "quality_metrics.csv"

print("QUALITY_CSV:", QUALITY_CSV)

IMAGE_ROOT: /Users/olga/MetaLogic/inputs | istnieje: True
QUALITY_CSV: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv


### 01 – Importy + ścieżki + lista obrazów

In [4]:
# %%
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image

DIR_DATA = Path("inputs")

def list_images(dir_path: Path):
    exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
    return sorted([p for p in dir_path.iterdir() if p.suffix.lower() in exts])

image_paths = list_images(DIR_DATA)
print("Liczba obrazów:", len(image_paths))

Liczba obrazów: 74


### 02 – Metryki jakości

In [5]:
# %%
def load_cv2_gray(path: Path):
    """
    Wczytuje obraz w skali szarości przez OpenCV.

    Zwraca:
    - tablicę uint8 (H, W) jeśli się udało,
    - None, jeśli OpenCV nie potrafi wczytać pliku (np. nietypowy TIFF/WEBP).
    """
    try:
        img_gray = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    except Exception as e:
        print(f"[QUALITY] Błąd odczytu pliku {path.name}: {e}")
        return None

    if img_gray is None:
        print(f"[QUALITY] OpenCV nie wczytał pliku (None): {path.name}")
        return None

    return img_gray


def blur_score(img_gray):
    return float(cv2.Laplacian(img_gray, cv2.CV_64F).var())


def contrast_score(img_gray):
    return float(img_gray.std())


def brightness_score(img_gray):
    return float(img_gray.mean())


def compute_quality(path: Path):
    """
    Liczy metryki jakości dla pojedynczego obrazu.

    Zwraca dict:
    - file_path, blur, contrast, brightness
    lub None, jeśli obraz nie został wczytany.
    """
    img_gray = load_cv2_gray(path)
    if img_gray is None:
        return None

    return {
        "file_path": str(path),
        "blur": blur_score(img_gray),
        "contrast": contrast_score(img_gray),
        "brightness": brightness_score(img_gray),
    }

### 03 – Batch

In [13]:
# %%
records = []

for i, p in enumerate(image_paths, start=1):
    print(f"[{i}/{len(image_paths)}] {p.name}")
    rec = compute_quality(p)
    if rec:
        records.append(rec)

df_quality = pd.DataFrame(records)
df_quality



[1/74] 0004.jpg
[2/74] 0006.jpg
[3/74] 0009.jpg
[4/74] 0022.jpg
[5/74] 0034.jpg
[6/74] 0043.jpg
[7/74] 0044.jpg
[8/74] 0074.jpg
[9/74] 0075.jpg
[10/74] 0077.jpg
[11/74] 0106.jpg
[12/74] 0109.jpg
[13/74] 0111.jpg
[14/74] 0112.jpg
[15/74] 0114.jpg
[16/74] 0115 2.jpg
[17/74] 0115.jpg
[18/74] 0116.jpg
[19/74] 0117.jpg
[20/74] 0119.jpg
[21/74] 0120.jpg
[22/74] 0121.jpg
[23/74] 0124 2.jpg
[24/74] 0124.jpg
[25/74] 0125.jpg
[26/74] 0126.jpg
[27/74] 0127.jpg
[28/74] 0128.jpg
[29/74] 0135.jpg
[30/74] 0136.jpg
[31/74] 0138.jpg
[32/74] 0143.jpg
[33/74] 0145.jpg
[34/74] 0161.jpg
[35/74] 0165.jpg
[36/74] 0171.jpg
[37/74] 0174.jpg
[38/74] 0175.jpg
[39/74] 0187 2.jpg
[40/74] 0187.jpg
[41/74] 0188.jpg
[42/74] 0189.jpg
[43/74] 0196.jpg
[44/74] 0203.jpg
[45/74] 0204.jpg
[46/74] 0205.jpg
[47/74] 0206.jpg
[48/74] 0211.jpg
[49/74] 0223.jpg
[50/74] 0226.jpg
[51/74] 0229.jpg
[52/74] 0233.jpg
[53/74] 0241.jpg
[54/74] 0242.jpg
[55/74] 0253.jpg
[56/74] 0256.jpg
[57/74] 0275.jpg
[58/74] 0282.jpg
[59/74] 0286.jpg


,file_path,blur,contrast,brightness
0,inputs/0004.jpg,217.266330,84.594967,94.090657
1,inputs/0006.jpg,1292.408395,61.694697,136.716768
2,inputs/0009.jpg,1598.740746,70.362099,119.924682
3,inputs/0022.jpg,1313.585982,60.515637,112.491175
4,inputs/0034.jpg,769.719032,69.227966,146.110244
...,...,...,...,...
69,inputs/default.jpg,183.868837,68.126906,124.180634
70,inputs/djjefault-6.jpg,207.015941,52.763417,114.342822
71,inputs/fg.jpg,368.405046,65.225553,117.281366
72,inputs/hhdefault-4.jpg,109.300670,63.062448,118.613073


### CSV

In [14]:
# %%
out_path = Path("outputs/quality_metrics.csv")
df_quality.to_csv(QUALITY_CSV, index=False)
print("Zapisano metryki jakości do:", QUALITY_CSV)

Zapisano metryki jakości do: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv


### 1. Sortowanie zdjęć po ostrości (blur)

In [15]:
# %%
"""
Sortowanie zdjęć po ostrości (blur).

Tworzy dwie tabele:
- df_sharpest  – zdjęcia od najostrzejszych
- df_blurriest – zdjęcia od najbardziej rozmytych
"""

df_sharpest = df_quality.sort_values("blur", ascending=False).reset_index(drop=True)
df_blurriest = df_quality.sort_values("blur", ascending=True).reset_index(drop=True)

print("Najostrzejsze 10 zdjęć:")
df_sharpest.head(10)

Najostrzejsze 10 zdjęć:


,file_path,blur,contrast,brightness
0,inputs/0117.jpg,3991.757403,84.260545,141.577361
1,inputs/0119.jpg,3535.061055,55.277084,126.753168
2,inputs/0233.jpg,2040.941630,59.834598,119.076823
3,inputs/0242.jpg,1805.842030,56.525427,115.524981
4,inputs/0241.jpg,1794.293993,49.054932,113.971275
5,inputs/0121.jpg,1733.936451,72.261555,112.207409
6,inputs/0009.jpg,1598.740746,70.362099,119.924682
7,inputs/0077.jpg,1401.519361,57.138214,102.661564
8,inputs/0022.jpg,1313.585982,60.515637,112.491175
9,inputs/0006.jpg,1292.408395,61.694697,136.716768


In [16]:
# %%
print("Najbardziej rozmyte 10 zdjęć:")
df_blurriest.head(10)

Najbardziej rozmyte 10 zdjęć:


,file_path,blur,contrast,brightness
0,inputs/0226.jpg,41.226857,95.203951,172.202915
1,inputs/0188.jpg,74.551545,57.136059,180.875610
2,inputs/0136.jpg,77.113161,61.731172,135.309397
3,inputs/default-4.jpg,86.640338,83.075988,80.469344
4,inputs/0125.jpg,105.324313,58.087298,173.330123
5,inputs/0074.jpg,107.961643,64.880026,116.425343
6,inputs/hhdefault-4.jpg,109.300670,63.062448,118.613073
7,inputs/0161.jpg,110.241569,60.878537,157.209427
8,inputs/default-5.jpg,112.878092,86.334072,119.070590
9,inputs/0124.jpg,115.316205,66.185200,131.595816


### 2. Filtrowanie „najgorszych” jakościowo (np. 10% najbardziej rozmytych)

In [17]:
# %%
"""
Wybór najgorszych jakościowo zdjęć pod względem rozmycia.

Wyznacza próg jako 10. percentyl wartości 'blur'
i zwraca wszystkie zdjęcia poniżej tego progu.
"""

blur_threshold = df_quality["blur"].quantile(0.10)
print("Próg blur (10% najbardziej rozmyte):", blur_threshold)

df_worst_blur = df_quality[df_quality["blur"] <= blur_threshold].copy()
df_worst_blur = df_worst_blur.sort_values("blur", ascending=True).reset_index(drop=True)

print("Liczba zdjęć uznanych za najbardziej rozmyte:", len(df_worst_blur))
df_worst_blur

Próg blur (10% najbardziej rozmyte): 111.03252606582004
Liczba zdjęć uznanych za najbardziej rozmyte: 8


,file_path,blur,contrast,brightness
0,inputs/0226.jpg,41.226857,95.203951,172.202915
1,inputs/0188.jpg,74.551545,57.136059,180.875610
2,inputs/0136.jpg,77.113161,61.731172,135.309397
3,inputs/default-4.jpg,86.640338,83.075988,80.469344
4,inputs/0125.jpg,105.324313,58.087298,173.330123
5,inputs/0074.jpg,107.961643,64.880026,116.425343
6,inputs/hhdefault-4.jpg,109.300670,63.062448,118.613073
7,inputs/0161.jpg,110.241569,60.878537,157.209427
